In [4]:
import os
import numpy as np
#from prototype import prototype
#import scipy.spatial.distance as ssd
#from dissimilarity import dissimilarity
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

In [5]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.datasets import cifar10
from PIL import Image

In [6]:
train=cifar10.load_data()[0][0]
train_label=cifar10.load_data()[0][1]
x_test=cifar10.load_data()[1][0]
y_test=cifar10.load_data()[1][1]
y_test=y_test.reshape(len(y_test))
train_label=train_label.reshape(len(train_label))
C=3

Defined histogram

In [9]:
for value in range(8):
    entropy=value+1
    area=2**entropy
    Q=int(train.shape[1]*train.shape[2]/area)
    mod=train.shape[1]*train.shape[2]%area
    pixel_set=[]
    for n in range(area):
        if n < mod:
            for i in range(Q+1):
                pixel_set.append(n)
        else:
            for i in range(Q):
                pixel_set.append(n)
    equalized=np.zeros([train.shape[0],train.shape[1]*train.shape[2],C])
    a=train.reshape(-1,train.shape[1]*train.shape[2],C)
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set)):
        equalized[ind2==i]=int(pixel_set[i])
    equalized=equalized.reshape(train.shape[0],train.shape[1],train.shape[2],C)
    equalized_test=np.zeros([x_test.shape[0],x_test.shape[1]*x_test.shape[2],C])
    a=x_test.reshape(-1,x_test.shape[1]*x_test.shape[2],C)
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set)):
        equalized_test[ind2==i]=int(pixel_set[i])
    equalized_test=equalized_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],C)
    makefile(equalized,"Data/CIFAR10_edit/x_train"+str(entropy)+".pkl")
    makefile(equalized_test,"Data/CIFAR10_edit/x_test"+str(entropy)+".pkl")

Average CIFAR10

In [3]:
train=cifar10.load_data()[0][0]
train_label=cifar10.load_data()[0][1]
x_test=cifar10.load_data()[1][0]
y_test=cifar10.load_data()[1][1]
y_test=y_test.reshape(len(y_test))
train_label=train_label.reshape(len(train_label))
C=3
for label in range(10):
    print(label)
    x_train=train[train_label==label]
    pixel_set_R=[]
    pixel_set_G=[]
    pixel_set_B=[]
    for c in range(C):
        pixel_list=dict()
        for n in range(256):
            pixel_list[n]=0
        for data in x_train:
            for raw in data:
                for pixel in raw:
                    pixel_list[pixel[c]]+=1
        hist_list=dict()
        for num in pixel_list:
            hist_list[num]=pixel_list[num]/len(x_train)
        hist_num=dict()
        residual=0
        for num in hist_list:
            hist_num[num]=int(np.round(hist_list[num]+residual,0))
            residual=hist_list[num]+residual-int(np.round(hist_list[num]+residual,0))

        if c==0:
            for num in hist_num:
                repeat=hist_num[num]
                for n in range(repeat):
                    pixel_set_R.append(num)
        elif c==1:
            for num in hist_num:
                repeat=hist_num[num]
                for n in range(repeat):
                    pixel_set_G.append(num)
        else:
            for num in hist_num:
                repeat=hist_num[num]
                for n in range(repeat):
                    pixel_set_B.append(num)
    red=np.zeros([train.shape[0],train.shape[1]*train.shape[2]])
    green=np.zeros([train.shape[0],train.shape[1]*train.shape[2]])
    blue=np.zeros([train.shape[0],train.shape[1]*train.shape[2]])

    a=train[:,:,:,0].reshape(-1,train.shape[1]*train.shape[2])
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set_R)):
        red[ind2==i]=int(pixel_set_R[i])

    a=train[:,:,:,1].reshape(-1,train.shape[1]*train.shape[2])
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set_G)):
        green[ind2==i]=int(pixel_set_G[i])
    a=train[:,:,:,2].reshape(-1,train.shape[1]*train.shape[2])
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set_B)):
        blue[ind2==i]=int(pixel_set_B[i])
    equalized=np.concatenate([red.reshape(train.shape[0],train.shape[1],train.shape[2],1),green.reshape(train.shape[0],train.shape[1],train.shape[2],1),blue.reshape(train.shape[0],train.shape[1],train.shape[2],1)], axis=3)

    makefile(equalized,"Data/CIFAR10_edit/x_train_average_"+str(label)+".pkl")
    red=np.zeros([x_test.shape[0],x_test.shape[1]*x_test.shape[2]])
    green=np.zeros([x_test.shape[0],x_test.shape[1]*x_test.shape[2]])
    blue=np.zeros([x_test.shape[0],x_test.shape[1]*x_test.shape[2]])

    a=x_test[:,:,:,0].reshape(-1,x_test.shape[1]*x_test.shape[2])
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set_R)):
        red[ind2==i]=int(pixel_set_R[i])

    a=x_test[:,:,:,1].reshape(-1,x_test.shape[1]*x_test.shape[2])
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set_G)):
        green[ind2==i]=int(pixel_set_G[i])
    a=x_test[:,:,:,2].reshape(-1,x_test.shape[1]*x_test.shape[2])
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set_B)):
        blue[ind2==i]=int(pixel_set_B[i])
    equalized_test=np.concatenate([red.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1),green.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1),blue.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)], axis=3)
    makefile(equalized_test,"Data/CIFAR10_edit/x_test_average_"+str(label)+".pkl")

Average all

In [ ]:
train=cifar10.load_data()[0][0]
train_label=cifar10.load_data()[0][1]
x_test=cifar10.load_data()[1][0]
y_test=cifar10.load_data()[1][1]
y_test=y_test.reshape(len(y_test))
train_label=train_label.reshape(len(train_label))
C=3

x_train=train
pixel_set_R=[]
pixel_set_G=[]
pixel_set_B=[]
for c in range(C):
    pixel_list=dict()
    for n in range(256):
        pixel_list[n]=0
    for data in x_train:
        for raw in data:
            for pixel in raw:
                pixel_list[pixel[c]]+=1
    hist_list=dict()
    for num in pixel_list:
        hist_list[num]=pixel_list[num]/len(x_train)
    hist_num=dict()
    residual=0
    for num in hist_list:
        hist_num[num]=int(np.round(hist_list[num]+residual,0))
        residual=hist_list[num]+residual-int(np.round(hist_list[num]+residual,0))

    if c==0:
        for num in hist_num:
            repeat=hist_num[num]
            for n in range(repeat):
                pixel_set_R.append(num)
    elif c==1:
        for num in hist_num:
            repeat=hist_num[num]
            for n in range(repeat):
                pixel_set_G.append(num)
    else:
        for num in hist_num:
            repeat=hist_num[num]
            for n in range(repeat):
                pixel_set_B.append(num)
red=np.zeros([train.shape[0],train.shape[1]*train.shape[2]])
green=np.zeros([train.shape[0],train.shape[1]*train.shape[2]])
blue=np.zeros([train.shape[0],train.shape[1]*train.shape[2]])

a=train[:,:,:,0].reshape(-1,train.shape[1]*train.shape[2])
ind=np.argsort(a,axis=1,kind="mergesort")
ind2=np.argsort(ind,axis=1)
for i in range(len(pixel_set_R)):
    red[ind2==i]=int(pixel_set_R[i])

a=train[:,:,:,1].reshape(-1,train.shape[1]*train.shape[2])
ind=np.argsort(a,axis=1,kind="mergesort")
ind2=np.argsort(ind,axis=1)
for i in range(len(pixel_set_G)):
    green[ind2==i]=int(pixel_set_G[i])
a=train[:,:,:,2].reshape(-1,train.shape[1]*train.shape[2])
ind=np.argsort(a,axis=1,kind="mergesort")
ind2=np.argsort(ind,axis=1)
for i in range(len(pixel_set_B)):
    blue[ind2==i]=int(pixel_set_B[i])
equalized=np.concatenate([red.reshape(train.shape[0],train.shape[1],train.shape[2],1),green.reshape(train.shape[0],train.shape[1],train.shape[2],1),blue.reshape(train.shape[0],train.shape[1],train.shape[2],1)], axis=3)

makefile(equalized,"Data/CIFAR10_edit/x_train_average_all.pkl")
red=np.zeros([x_test.shape[0],x_test.shape[1]*x_test.shape[2]])
green=np.zeros([x_test.shape[0],x_test.shape[1]*x_test.shape[2]])
blue=np.zeros([x_test.shape[0],x_test.shape[1]*x_test.shape[2]])

a=x_test[:,:,:,0].reshape(-1,x_test.shape[1]*x_test.shape[2])
ind=np.argsort(a,axis=1,kind="mergesort")
ind2=np.argsort(ind,axis=1)
for i in range(len(pixel_set_R)):
    red[ind2==i]=int(pixel_set_R[i])

a=x_test[:,:,:,1].reshape(-1,x_test.shape[1]*x_test.shape[2])
ind=np.argsort(a,axis=1,kind="mergesort")
ind2=np.argsort(ind,axis=1)
for i in range(len(pixel_set_G)):
    green[ind2==i]=int(pixel_set_G[i])
a=x_test[:,:,:,2].reshape(-1,x_test.shape[1]*x_test.shape[2])
ind=np.argsort(a,axis=1,kind="mergesort")
ind2=np.argsort(ind,axis=1)
for i in range(len(pixel_set_B)):
    blue[ind2==i]=int(pixel_set_B[i])
equalized_test=np.concatenate([red.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1),green.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1),blue.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)], axis=3)
makefile(equalized_test,"Data/CIFAR10_edit/x_test_average_all.pkl")